In [70]:
import string
import numpy as np

In [71]:
def one_hot(n_classes: int, idx: int) -> np.ndarray:
    embedding = np.zeros(n_classes)
    embedding[idx] = 1.0
    return embedding


def word2seq(word: str, alphabet: list) -> np.ndarray:
    return np.array([one_hot(len(alphabet), alphabet.index(c)) for c in word])


def seq2word(seq: np.ndarray, alphabet: list) -> str:
    return ''.join([alphabet[np.argmax(x)] for x in seq])

In [72]:
alphabet = string.ascii_letters + " " + string.punctuation + string.digits
n_symbols = len(alphabet)
print(n_symbols, alphabet)

95 abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~0123456789


In [73]:
def softmax(x: np.ndarray) -> np.ndarray:
    y = np.exp(x)
    return y / np.sum(y)

In [74]:
def calculate_attention(states: np.ndarray, context: np.ndarray) -> np.ndarray:
    scores = softmax(states @ context)
    values = scores.reshape((scores.size, 1)) * states
    return np.sum(values, axis=0)

In [75]:
class EncoderRNN:
    def __init__(self, n_in: int, n_h: int) -> None:
        self.U: np.ndarray = np.random.uniform(-1, 1, (n_h, n_in))
        self.V: np.ndarray = np.random.uniform(-1, 1, (n_h, n_h))
        self.d: np.ndarray = np.zeros(n_h)
    
    def forward(self, sequence: np.ndarray) -> tuple[np.ndarray]:
        states: np.ndarray = np.zeros((len(sequence), self.d.size))
        context: np.ndarray = np.zeros(len(self.V))

        for i,x in enumerate(sequence):
            context = np.tanh(self.U @ x + self.V @ context + self.d)
            states[i] = context.copy()
        
        return (states, context)

In [76]:
class DecoderRNN:
    def __init__(self, n_h: int, n_out: int) -> None:
        self.V: np.ndarray = np.random.uniform(-1, 1, (n_h, n_h))
        self.d: np.ndarray = np.zeros(n_h)

        self.W: np.ndarray = np.random.uniform(-1, 1, (n_out, 2*n_h))
        self.b: np.ndarray = np.zeros(n_out)
    
    def forward(self, states: np.ndarray, context: np.ndarray, t: int) -> np.ndarray:
        outputs = np.zeros((t, self.b.size))

        for i in range(t):
            context = np.tanh(self.V @ context + self.d)
            attention = calculate_attention(states, context)
            ctx_att = np.concat((context, attention), axis=0)
            outputs[i] = softmax(self.W @ ctx_att + self.b)
            
        return outputs

In [82]:
n_h = 32

encoder = EncoderRNN(n_symbols, n_h)
decoder = DecoderRNN(n_h, n_symbols)

in_text = 'the quick brown fox jumps over the lazy dog'
x = word2seq(in_text, alphabet)

states, context = encoder.forward(x)
output = decoder.forward(states, context, 128)

out_text = seq2word(output, alphabet)
print(out_text)

kA~ufRJuDdVwV:~nB+:~\GF<'f+<bz;^GfMSVF^5t]RcfG5/F^QtS vwVcR5$G!F5QvO5Gw]ysesR yt4GBtZ '/?' G-tJFQfT/J^SF]6vwVHf~lQ}VCqOf;fSSJ'?5
